In [1]:
def mean_std_func(tt):

    xx = tt['Оценка студента (опрос)'].describe()
    mean_data = xx['mean']
    std_data = xx['std']
    moda_data = stats.mode(tt['Оценка студента (опрос)'])
    moda = float(moda_data.mode)
    
    return [mean_data, std_data, moda]

In [2]:
def generator_trunc_norm(moda, std, dataframe):
    
    dataframe.dropna(subset = ['Оценка студента (опрос)'])
    flag2 = True

    myclip_a = 1
    myclip_b = 5
    a = (myclip_a - moda) / std 
    b = (myclip_b - moda) / std
    
    for u in dataframe['Группа'].unique().tolist():
        for y in dataframe['Студент (Код)'].unique().tolist():
            x_stud = dataframe.loc[(dataframe['Студент (Код)'] == y) & (dataframe['Группа'] == u)].sort_values(by = ['Оценка студента (опрос)'], ascending = False)

            if len(x_stud) > 0:
                rand_trunc_norm = truncnorm.rvs(a, b, loc = moda, scale = std, size = len(x_stud))
            else:
                continue
            list_of_rand = []

            for j in rand_trunc_norm:
                list_of_rand.append(round(j, 1))

            list_of_rand.sort(reverse=True)
            x_stud['truncnorm'] = list_of_rand

            if flag2 == True:
                flag2 = False
                empty_x_stud = dataframe.iloc[0:0]
                global_x_stud = pd.concat([x_stud, empty_x_stud])
            else:
                global_x_stud = pd.concat([x_stud, global_x_stud])
    
    return global_x_stud


In [3]:
def Moda(df, column):
    dict = {}
    for x in df[column]:
        if x not in dict:
            dict[x] = 1
        else:
            dict[x] += 1
    maxp = 0
    moda = None
    for i in dict:
        if dict[i] > maxp:
            maxp = dict[i]
            moda = i
    return moda


In [4]:
def round_data_hyp_1(data, column):
    
    data_round = data.copy()
    data_round['hypotez_1_round'] = round(data_round[column])
    
    return data_round

In [5]:
#global_x_stud.loc[(global_x_stud['Студент (Код)'] == 281) & (global_x_stud['Группа'] == 9) & (global_x_stud['Семестр'] == 1)]

In [6]:
def rating_data_trunc(data):
    
    rating = data.drop(columns = ['Группа','Семестр', 'Оценка студента (опрос)', 'Студент (Код)']).groupby(['Дисциплина']).mean().sort_values(by = ['truncnorm'], ascending=False)
    
    return rating

In [7]:
def rating_old(data_old):
    
    rating_old = data_old.dropna(subset = ['Оценка студента (опрос)']).drop(columns = ['Группа','Семестр', 'Студент (Код)']).groupby(['Дисциплина']).mean().sort_values(by = ['Оценка студента (опрос)'], ascending=False)
    
    return rating_old

In [8]:
def hypotez_2(data, column):
    
    data.dropna(subset = [column])
    flag3 = True
    for z in data['Группа'].unique().tolist():
        for x in data['Студент (Код)'].unique().tolist():
            for c in data['Семестр'].unique().tolist():
                sem_stud = data.loc[(data['Студент (Код)'] == x) & (data['Группа'] == z) & (data['Семестр'] == c)].sort_values(by = [column], ascending = False)
                if len(sem_stud) > 0:

                    list_of_trunc = sem_stud[column].tolist()
                    list_of_trunc.sort(reverse=True)
                    high_numb = list_of_trunc[0]
                    raznost = 5 - high_numb
                    for i in range(len(list_of_trunc)):
                        abc = list_of_trunc[i] + raznost
                        list_of_trunc[i] = abc
                        if list_of_trunc[i] > 5:
                            list_of_trunc[i] = 5
                            
                    sem_stud['hypotez_2'] = list_of_trunc

                    if flag3 == True:
                        flag3 = False
                        empty_x_stud = data.iloc[0:0]
                        global_x_stud_new = pd.concat([sem_stud, empty_x_stud])
                    else:
                        global_x_stud_new = pd.concat([sem_stud, global_x_stud_new])
                else:
                    continue
    
    return global_x_stud_new

In [9]:
#data_hyp_2.loc[(data_hyp_2['Студент (Код)'] == 271) & (data_hyp_2['Группа'] == 9) & (data_hyp_2['Семестр'] == 4)]

In [10]:
def data_hyp_2_round(data_hyp_2):
    
    data_hyp_2_round = data_hyp_2.copy()
    data_hyp_2_round['hypotez_2_round'] = round(data_hyp_2_round['hypotez_2'])
    
    return data_hyp_2_round

In [11]:
#rating_by_students_3 = global_x_stud_new.drop(columns = ['Группа','Семестр', 'Оценка студента (опрос)', 'Студент (Код)', 'truncnorm']).groupby(['Дисциплина']).mean().sort_values(by = ['truncnorm_new'], ascending=False)
#rating_by_students_4 = global_x_stud_new_round.drop(columns = ['Группа','Семестр', 'Оценка студента (опрос)', 'Студент (Код)', 'truncnorm']).groupby(['Дисциплина']).mean().sort_values(by = ['truncnorm_new'], ascending=False)

In [12]:
def graph_rating(data, column):
    
    rating = data.loc[:,data.columns.isin([column,'Дисциплина'])].groupby(['Дисциплина']).mean().sort_values(by = column, ascending=False)
    rating_ = rating.reset_index()
    labels = rating_['Дисциплина'].round(1).to_list()
    men_means = rating_[column].round(2).to_list()
    x = np.arange(len(labels)) 
    width = 0.4  
    fig, ax = plt.subplots(figsize=(21,10))
    rects1 = ax.bar(x - width/2, men_means, width, label=column)


    ax.set(ylabel='Оценка', ylim=(3, 5.5))
    ax.set_title(column)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.set_xlabel("Номер дисциплины", labelpad=20, weight='bold', size=12)
    ax.legend()

    fig.tight_layout()

    return plt.show()


In [13]:
def method_lowest_squares(data, data_rating, column_rating):
    
    data_rating.dropna(subset = [column_rating])
    rat_2 = data_rating.loc[:,data_rating.columns.isin([column_rating,'Дисциплина'])].groupby(['Дисциплина']).mean().sort_values(by = column_rating, ascending=False)
    rat_2.reset_index(inplace=True)
    flag_x = True
    for a in data['Группа'].unique().tolist():
        for b in data['Студент (Код)'].unique().tolist():              
            test_data = data.loc[(data['Студент (Код)'] == b) & (data['Группа'] == a)]
            if len(test_data) > 0:
                test_disc = test_data['Дисциплина'].to_numpy()
                test_trunc = test_data['truncnorm'].to_numpy()

                sum_y_i = 0
                for i in test_disc:

                    sum_y_i += float(rat_2.loc[rat_2['Дисциплина'] == i][column_rating].to_numpy())

                m_k = len(test_data)
                sum_x_i_k = test_data['truncnorm'].sum()
                x_sh_k = (1/m_k)*sum_x_i_k
                y_sh = (1/m_k)*sum_y_i

                sum_razn_x_v2 = 0
                for j in test_trunc:
                    sum_razn_x_v2 += (j - x_sh_k) ** 2

                sum_razn_x_razn_y = 0
                for i in test_disc: 
                    y_i = float(rat_2.loc[rat_2['Дисциплина'] == i][column_rating].to_numpy())

                    for g in range(len(test_data.loc[test_data['Дисциплина'] == i]['truncnorm'])):

                        x_i_k = test_data.loc[test_data['Дисциплина'] == i]['truncnorm'].to_numpy()[g]
                        sum_razn_x_razn_y += (x_i_k - x_sh_k) * (y_i - y_sh)

                C_xx_k = (1/m_k) * sum_razn_x_v2
                C_xy_k = (1/m_k) * sum_razn_x_razn_y
                L_k = C_xy_k / C_xx_k
                B_k = y_sh - L_k * x_sh_k

                test_data['new_method'] = L_k * test_data['truncnorm'] + B_k
            
                if flag_x == True:
                    flag_x = False
                    empty_x = test_data.iloc[0:0]
                    global_x = pd.concat([test_data, empty_x])
                else:
                    global_x = pd.concat([test_data, global_x])
            else:
                continue    
    
    return global_x

In [14]:
def graph_old_rating(t_empty, t_empty_sep):
    
    df_ = t_empty.loc[:,t_empty.columns.isin(['Оценка студента (опрос)','Дисциплина'])].groupby(['Дисциплина']).mean()
    df_sep = t_empty_sep.loc[:,t_empty.columns.isin(['Оценка студента (опрос)','Дисциплина'])].groupby(['Дисциплина']).mean()
    merge_df = df_.merge(df_sep, on="Дисциплина")
    merge_df['Квадрат разности'] = (merge_df['Оценка студента (опрос)_x'] - merge_df['Оценка студента (опрос)_y']) ** 2
    izmen_data = merge_df.loc[merge_df['Квадрат разности'] != 0]
    izmen_data.sort_values(by = ['Оценка студента (опрос)_y'], ascending = False, inplace = True)
    izmen_data.reset_index(inplace=True)
    izmen_data.sort_values(by = ['Оценка студента (опрос)_x'], ascending = False, inplace = True)
    labels = izmen_data['Дисциплина'].round(1).to_list()
    men_means = izmen_data['Оценка студента (опрос)_x'].round(2).to_list()

    x = np.arange(len(labels)) 
    width = 0.2  

    fig, ax = plt.subplots(figsize=(21,10))
    rects1 = ax.bar(x - width/2, men_means, width, label='Рейтинг дисциплин после заполнения пропусков')

    ax.set(ylabel='Оценка', ylim=(3, 5.5))
    ax.set_title('Рейтинг дисциплин')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()


    def autolabel(rects):
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{}'.format(height),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')


    autolabel(rects1)


    fig.tight_layout()

    plt.show()

In [44]:
def graph_old(t_empty, column):
    
    rating = t_empty.loc[:,t_empty.columns.isin([column,'Дисциплина'])].groupby(['Дисциплина']).mean().sort_values(by = column, ascending=False)
    rating_ = rating.reset_index()
    labels = rating_['Дисциплина'].round(1).to_list()
    men_means = rating_['Оценка студента (опрос)'].round(2).to_list()

    x = np.arange(len(labels)) 
    width = 0.4  

    fig, ax = plt.subplots(figsize=(21,10))
    rects1 = ax.bar(x - width/2, men_means, width, label='Рейтинг дисциплин после заполнения пропусков')

    ax.set(ylabel='Оценка', ylim=(2, 5.5))
    ax.set_title('Рейтинг дисциплин')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()

    plt.show()

In [16]:
#rating_ = rating_by_students_1.sort_values(by = ['Оценка студента (опрос)'])
#rating_

In [17]:
#graph_rating(rating_by_students_1.sort_values(by = []))

In [18]:
#rating_by_students_1.plot(kind='barh')

In [19]:
"""ax = rating_.plot(kind='barh', figsize=(16, 50), zorder=2, width=0.85)

# Despine
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

# Switch off ticks
ax.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

# Draw vertical axis lines
vals = ax.get_xticks()
for tick in vals:
    ax.axvline(x=tick, linestyle='dashed', alpha=0.4, zorder=1)

# Set x-axis label
ax.set_xlabel("Оценка", labelpad=20, weight='bold', size=12)

# Set y-axis label
ax.set_ylabel("Номер дисциплины", labelpad=20, weight='bold', size=12)
"""


'ax = rating_.plot(kind=\'barh\', figsize=(16, 50), zorder=2, width=0.85)\n\n# Despine\nax.spines[\'right\'].set_visible(False)\nax.spines[\'top\'].set_visible(False)\nax.spines[\'left\'].set_visible(False)\nax.spines[\'bottom\'].set_visible(False)\n\n# Switch off ticks\nax.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")\n\n# Draw vertical axis lines\nvals = ax.get_xticks()\nfor tick in vals:\n    ax.axvline(x=tick, linestyle=\'dashed\', alpha=0.4, zorder=1)\n\n# Set x-axis label\nax.set_xlabel("Оценка", labelpad=20, weight=\'bold\', size=12)\n\n# Set y-axis label\nax.set_ylabel("Номер дисциплины", labelpad=20, weight=\'bold\', size=12)\n'

In [20]:
#larg_20 = rating_.nlargest(25, columns = ['Оценка студента (опрос)'])

In [21]:
#graph_rating(larg_20)

In [22]:
#global_x_stud_new_round

In [23]:
#global_x_stud_new_round.loc[(global_x_stud_new_round['Студент (Код)'] == 3) & (global_x_stud_new_round['Группа'] == 1) & (global_x_stud_new_round['Семестр'] == 7)]

In [24]:
#rat_2 = rating_by_students_2.reset_index()
#rat_2['Ранг дисциплины'] = rat_2['truncnorm'].rank(method='dense', ascending=False)

In [25]:
#rat_2

In [26]:
#rating_by_students_2.loc[(rating_by_students_2['Студент (Код)'] == b) & (rating_by_students_2['Группа'] == a) & (rating_by_students_2['Семестр'] == c)]

In [27]:
#rating_by_students_4

In [28]:
#rating_by_students_2

In [29]:
#larg_20_2 = rating_by_students_2.sort_values(by = ['truncnorm']).nlargest(25, columns = ['truncnorm'])
#larg_20_4 = rating_by_students_4.sort_values(by = ['truncnorm_new']).nlargest(25, columns = ['truncnorm_new'])

In [30]:
#graph_rating(larg_20_2)

In [31]:
#graph_rating(larg_20_4)

In [32]:
#small_20_4 = rating_by_students_4.sort_values(by = ['truncnorm_new']).nsmallest(30, columns = ['truncnorm_new'])
#graph_rating(small_20_4)

In [33]:
#small_20_2 = rating_by_students_2.sort_values(by = ['truncnorm']).nsmallest(30, columns = ['truncnorm'])
#graph_rating(small_20_2)

In [34]:
#rating_by_students_2

In [35]:
#small_20_1 = rating_by_students_1.sort_values(by = ['Оценка студента (опрос)']).nsmallest(30, columns = ['Оценка студента (опрос)'])
#graph_rating(small_20_1)

In [36]:
#larg_20_1 = rating_by_students_1.sort_values(by = ['Оценка студента (опрос)']).nlargest(30, columns = ['Оценка студента (опрос)'])
#graph_rating(larg_20_1)

In [37]:
#rat_2 = rating_by_students_2.reset_index()
#rat_2['Ранг дисциплины'] = rat_2['truncnorm'].rank(method='dense', ascending=False)

In [38]:
#np.isnan(test_data.loc[test_data['Дисциплина'] == 9]['truncnorm'].to_numpy()) == False

In [39]:
#tt = global_x_stud.loc[(global_x_stud['Студент (Код)'] == 271) & (global_x_stud['Группа'] == 9)]
#tt

In [40]:
#x = rat_2.loc[rat_2['Дисциплина'] == 108]
#x

In [41]:
#tt['truncnorm'].to_numpy()

In [42]:
#rat_2